<a href="https://colab.research.google.com/github/crisbpadilla/DataScience-M2M-course-Datatalent/blob/main/Bokeh_StackbarChart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bokeh

 Data procesing, filtering and creating a dataset to build up a stack bar chart. The idea is get a visualization of the voting of every state in USA


In [2]:
import pandas as pd

elections_results= pd.read_csv('1976-2016-president.csv')
elections_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3740 entries, 0 to 3739
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            3740 non-null   int64  
 1   state           3740 non-null   object 
 2   state_po        3740 non-null   object 
 3   state_fips      3740 non-null   int64  
 4   state_cen       3740 non-null   int64  
 5   state_ic        3740 non-null   int64  
 6   office          3740 non-null   object 
 7   candidate       3474 non-null   object 
 8   party           3404 non-null   object 
 9   writein         3740 non-null   bool   
 10  candidatevotes  3740 non-null   int64  
 11  totalvotes      3740 non-null   int64  
 12  version         3740 non-null   int64  
 13  notes           0 non-null      float64
dtypes: bool(1), float64(1), int64(7), object(5)
memory usage: 383.6+ KB


once opening the data, we could use the info to see the columns that we have in our archive,and we could select the columns that we will use in our future analysis

In [3]:
#elections_results.head()
# we can observe that certain values are useful to analyse data

columns=['year','state','candidate','party','candidatevotes','totalvotes']
# so we will open the documents back again but with the columns that we need
# using arguments of the method read_csv to filter it instead of working on the
# dataframe

data_elections=pd.read_csv('1976-2016-president.csv',usecols= columns,dtype={'year':str})

data_elections


,year,state,candidate,party,candidatevotes,totalvotes
0,1976,Alabama,"Carter, Jimmy",democrat,659170,1182850
1,1976,Alabama,"Ford, Gerald",republican,504070,1182850
2,1976,Alabama,"Maddox, Lester",american independent party,9198,1182850
3,1976,Alabama,"Bubar, Benjamin """"Ben""""",prohibition,6669,1182850
4,1976,Alabama,"Hall, Gus",communist party use,1954,1182850
...,...,...,...,...,...,...
3735,2016,Wyoming,NaN,NaN,6904,258788
3736,2016,Wyoming,NaN,independent,3224,258788
3737,2016,Wyoming,Blank Vote,NaN,2661,258788
3738,2016,Wyoming,"Castle, Darrell L.",constitution party,2042,258788


In [4]:
elections= data_elections[(data_elections['party']=='republican')|(data_elections['party']=='democrat')]
state_vote=elections.groupby('state')
alabama=state_vote.get_group('Alabama')
year= data_elections['year'].unique() # we got the dates as categorical values

parties=alabama['party'].unique()
party_votes = alabama.groupby('party')['candidatevotes']
data={party: party_votes.get_group(party)for party in parties}
data['year']=year
data

{'democrat': 0       659170
 333     636730
 697     551899
 1020    549506
 1305    690080
 1675    662165
 2043    692611
 2412    693933
 2729    813479
 3080    795696
 3396    729547
 Name: candidatevotes, dtype: int64,
 'republican': 1        504070
 332      654192
 696      872849
 1019     815576
 1304     804283
 1674     769044
 2042     941173
 2411    1176394
 2728    1266546
 3079    1255925
 3395    1318255
 Name: candidatevotes, dtype: int64,
 'year': array(['1976', '1980', '1984', '1988', '1992', '1996', '2000', '2004',
        '2008', '2012', '2016'], dtype=object)}

In [5]:
# I need a dictionary to use the key data to get the information from

year= data_elections['year'].unique() # we got the dates as categorical values
#we can do the same as the last coding doing it without any for loop as..
votes=alabama.groupby('party')['candidatevotes']
rep=votes.get_group('republican')
dem=votes.get_group('democrat')

source={'democrat':dem,
        'republican':rep,
        'year':year}

# The key in the dictionary must to match the value of the groups stack in the
# graph taken from the dataset

source

{'democrat': 0       659170
 333     636730
 697     551899
 1020    549506
 1305    690080
 1675    662165
 2043    692611
 2412    693933
 2729    813479
 3080    795696
 3396    729547
 Name: candidatevotes, dtype: int64,
 'republican': 1        504070
 332      654192
 696      872849
 1019     815576
 1304     804283
 1674     769044
 2042     941173
 2411    1176394
 2728    1266546
 3079    1255925
 3395    1318255
 Name: candidatevotes, dtype: int64,
 'year': array(['1976', '1980', '1984', '1988', '1992', '1996', '2000', '2004',
        '2008', '2012', '2016'], dtype=object)}

In [6]:
from bokeh.plotting import figure,show,output_notebook
output_notebook()
# we create a figure

fig_stack=figure(x_range=year)
fig_stack.vbar_stack(parties,x='year', color=['red', 'blue'], width=0.8, source=source)
show(fig_stack)



## create a grid by state

In [23]:
from bokeh.layouts import gridplot
from bokeh.plotting import show,figure,output_notebook
from bokeh.models import HoverTool

output_notebook()

graph=[]
states=elections['state'].unique()
year= data_elections['year'].unique()
parties=elections['party'].unique()

for i in states:

  name_state= state_vote.get_group(i)

  party_votes = name_state.groupby('party')['candidatevotes']
  data={party: party_votes.get_group(party)for party in parties}
  data['year']=year

  fig_chart = figure(title=i, x_range= year,width=300,height=300,tooltips='$name total votes:@$name')
  fig_chart.vbar_stack(parties,x='year', color=['blue','red'],legend_label=list(parties), width=0.6, source=data)
  fig_chart.xaxis.major_label_orientation = 45  # Set angle in degrees
  fig_chart.legend.location='top_left'

  graph.append(fig_chart)


grid = gridplot(graph,ncols=3)
show(grid)